### Задание 2.2
Вычислите дисперсию для этого ряда, используя формулу дисперсии для дискретного либо интервального вариационного ряда.

In [28]:
import numpy as np
a=np.array([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2])
print(round(sum((a-np.mean(a))**2)/len(a),3))

print(round(np.var(a),3))

0.317
0.317


### Задание 2.7
* Подготовьте данные к классификации. Условно разделите вино на хорошее и плохое. Хорошим вином будем называть то, параметр quality которого — 6 и более.
* Разделите выборку на обучающую и тестовую в соотношении 70/30, в качестве значения параметра random_state возьмите число 42.
* Для начала обучите два классификатора: логистическую регрессию (с параметрами по умолчанию) и дерево решений (random_state = 42, максимальная глубина — 10).

wineQualityReds

In [50]:
import pandas as pd
import numpy as np
# Графика
import matplotlib.pyplot as plt
import seaborn as sns
# Деление выборки
from sklearn import model_selection

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble

from sklearn import metrics #метрики

In [2]:
wine_data = pd.read_csv('../../data/wineQualityReds.csv')
wine_data.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
# wine_data.info()

1. Пропусков нет
2. Признаки все числовые

In [11]:
wine_data['y'] = np.where((wine_data['quality'] >= 6), 1, 0)
 
# wine_data.insert(loc = wine_data.shape[1], column= 'temp', value = region)

# temp=wine_data.corr(numeric_only=True).round(2)
# #  фильтор
# filtr_1=np.tril(temp)

# fig = plt.figure(figsize=(10, 12))
# sns.heatmap(temp, # df
#             annot=True, # вывод надписей (подписать фон)
#             vmin=-1, vmax=1, center= 0,# граници тепловой шкалы
#             cmap= 'hsv',    # палитра цветов  https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html
#             linewidths=3, linecolor='black', # толщина и цвет оконтовки ячеек
#             cbar_kws= {'orientation': 'horizontal'},  # отображать цветовую шкалу горизонтально
#             # cbar=False,  # убрать цветовую шкалу
#             # square=True    # форма ячеек квадрат
#             mask=filtr_1  # фильтор выводимых значений
#             )# https://datastart.ru/blog/read/seaborn-heatmaps-13-sposobov-nastroit-vizualizaciyu-matricy-korrelyacii

# wine_data=wine_data.drop('temp',axis=1)

* Сильной кореляции нет

In [16]:
# sns.countplot(data=wine_data, x='y');

* Данные почти сбалонсированы

In [18]:
X = wine_data.drop(['quality','y'], axis=1)
y = wine_data['y']
y.mean()*100

53.47091932457786

In [28]:
# Формируем обучающую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42) # , stratify=y
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))
print(f'Баланс в Train выборке {y_train.mean()*100:.2f}')
print(f'Баланс в Test выборке {y_test.mean()*100:.2f}')

Train shape: (1119, 12)
Test shape: (480, 12)
Баланс в Train выборке 52.55
Баланс в Test выборке 55.62


In [47]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression()
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
# print("accuracy на валидационном наборе: {:.2f}".format(log_reg.score(X_train, y_train)))
# y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)

# acc_le=metrics.f1_score(y_train, y_train_pred)
# print('f1_score на Train:: {:.3f}'.format(acc_le))

acc_le=metrics.f1_score(y_test, y_test_pred)
print('f1_score на Test: {:.3f}'.format(acc_le))

f1_score на Test: 0.739


c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
#Создаем объект класса дерево решений
dt = tree.DecisionTreeClassifier(max_depth = 10, random_state=42)
#Обучаем дерево по алгоритму CARTrint
dt.fit(X_train, y_train)
#Выводим значения метрики 
# y_train_pred = dt.predict(X_train)
# print('Train: {:.3f}'.format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = dt.predict(X_test)
acc_le=metrics.f1_score(y_test, y_test_pred)
print('Test: {:.3f}'.format(acc_le))


Test: 0.760


### Задание 2.8
Обучите модель с использованием бэггинга (класс BaggingClassifier с random_state=42).

Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. Вычислите новое значение F1-score.

In [52]:
clf = ensemble.BaggingClassifier(estimator=dt, n_estimators=1500, random_state=42) # estimator=None == DecisionTreeClassifier
clf.fit(X_train, y_train)
y_test_pred = clf.predict(X_test)
acc_le=metrics.f1_score(y_test, y_test_pred)
print('Test: {:.3f}'.format(acc_le))

Test: 0.824
